In [1]:
from itertools import chain
from xpinyin import Pinyin
import os
import re
p = Pinyin()
def to_pinyin(s):
    return ''.join(chain.from_iterable(p.get_pinyin(s, tone_marks='numbers') ))

In [16]:
tex_name = 'Static11'           #材质贴图名称
font_name = 'MC20'              #字体名字

coorX = 0                       #起始坐标X
coorY = 2460                      #起始坐标Y

rectX = 20                      #字符宽度
rectY = 20                      #字符高度Y

row_num = 51                    #一行字符数

## 函数

In [9]:
def find_all_chara_text(a):     #输出并返回所有文本出现的单字、符号,以unicode编码排序 All_char.txt  该文档用于检视所有单字与编码检索
    f= open(a,encoding='utf-8')
    texts = f.read()
    f.close()

    font =[]
    for text in texts:
        if( ord(text)>3000 and text not in font):
            font.append(text)
    #font = sorted(font,key = to_pinyin)   #拼音排序
    font = sorted(font)                    #unicode编码排序

    #for item in font:
        #print(item,end='')
    print(f'total length:{len(font)}')

    all_char =''
    for item in font:
        all_char = all_char+item
    g = open(f'.\\files\\{font_name}all_char.txt','w',encoding='utf-8')
    g.write(all_char)
    g.close()
    return font

In [4]:
def output_font(text,row_num):  #单字输出到格式化文本 Draw_char.txt. Row_num 为一排字个数  该文档用于PS内绘制贴图
    out =''                                         
    for i in range(len(text)):
        if(i%row_num==0 and i!=0):
            out = out+'\n'+text[i]
        else:
            out = out+text[i]
    f = open(f'.\\files\\{font_name}draw_char.txt','w',encoding='utf-8')
    f.write(out)
    f.close()

In [5]:
name_lookup =[] 
def res_registration(font,tex_name,font_name,coorX,coorY,rectX,rectY,row_num):  #生成字库Res、字符编码对应集
    #name_lookup =[] 
    res =   (f'Texture {tex_name}\n'
            '{\n'
            f'filename = Media/Tex/{tex_name}.png\n'
            f'resgroup = 0\n'
            '}\n')
    coor = ''
    for i in range(len(font)):                                     
        character = font[i]
        #print(character)
        name = p.get_pinyin(character, tone_marks='numbers')        #获取字符拼音
        if ord(character)>65280 or ord(character) <12291:
            if name == '—':
                name = 'hengang'
            elif name == '‘':
                name = 'Ldanyinhao'
            elif name == '’':
                name = 'Rdanyinhao'
            elif name == '“':
                name = 'Lshuangyinhao'
            elif name == '”':
                name = 'Rshuangyinhao'
            elif name == '…':
                name = 'shenglvehao'
            elif name == '、':
                name = 'dunhao'
            elif name == '。':
                name = 'juhao'
            elif name == '！':
                name = 'gantanhao'
            elif name == '（':
                name = 'Lkuohao'
            elif name == '）':
                name = 'Rkuohao'
            elif name == '，':
                name = 'douhao'
            elif name == '：':
                name = 'maohao'
            elif name == '？':
                name = 'wenhao'

        j = 0
        while(1):
            fname = name+f'_{j}'
            if fname not in name_lookup:
                name_lookup.append(fname)
                break
            j += 1                              #获取字符唯一编码名

        if(i%row_num==0 and i!=0):
            coorY +=rectY
        coorX = (i%row_num)*rectX

        res =  res+(
            f"Animation {font_name}_{fname}\n"
            "{\n"
            f" texture = {tex_name}\n"
            f" rect = {coorX},{coorY},{rectX},{rectY}\n"
            f" hotspot = {rectX//2},{rectY//2}\n"
            f" zorder = 0.000000\n"
            f" resgroup = 0\n"
            f" frame = 1\n"
            "}\n"
        )                                           #打印Res
        coor = coor+fname+'\n'

    f = open(f'{tex_name}_{font_name}.res','w',encoding='utf-8')
    f.write(res)
    f.close()

    g = open(f'.\\files\\{font_name}lookup.txt','w',encoding='utf-8')
    g.write(coor)
    g.close()

In [6]:
def encode_gen(aa):               #通过chinese.txt生成编码后的english.txt
    ds = os.listdir('.\\files')
    i=0
    fonts_name =[]
    
    for d in ds:
        a = re.match('(.*?)lookup.txt',d)
        if(a):
            fonts_name.append(a.groups()[0])
            i+= 1
    for name in fonts_name:
        f= open(f'files\\{name}all_char.txt',encoding='utf-8')         
        locals()[f'{name}texts'] = f.read()
        f.close()
        g= open(f'files\\{name}lookup.txt',encoding='utf-8')         
        locals()[f'{name}lookup'] = g.read().split('\n')
        g.close()

    f= open(aa,encoding='utf-8')
    originals = f.read()
    f.close()
    
    trans =[]
    default_font_name = 'SH20'
    marker1 =0      #$检测
    counter =0      #*检测（字体套用范围）
    fontname =''
    font_name =  default_font_name
    for item in originals :
        if item =='$':
            marker1 =1
            continue
        if item =='*' and counter ==0:
            marker1 =0
            counter =1
            font_name = fontname
            continue
        if item =='*' and counter ==1:
            counter =0
            font_name = default_font_name
            fontname =''
            continue
        if marker1 ==1:
            fontname =fontname+ item
            continue
        texts = locals()[f'{font_name}texts'] 
        name_lookup = locals()[f'{font_name}lookup'] 
        if item in texts:
            index= name_lookup[texts.index(item)]           #查询单字index
            trans.append('{'+f'animation={font_name}_{index}'+'}')
        else:
            trans.append(item)                              #非汉字直接加入
    f= open('english.txt','w',encoding='utf-8')             #输出编码后english.txt
    for tran in trans:  
        f.write(tran)
    f.close()

## 运行

In [18]:
font = find_all_chara_text('a.txt')     

total length:69


In [19]:
output_font(font,row_num)

In [20]:
name_lookup =[] 
res_registration(font,tex_name,font_name,coorX,coorY,rectX,rectY,row_num)

In [23]:
encode_gen('chinese.txt')

# 测试
